### Copyright (C) Microsoft Corporation.  
see license.md file for Enterprise Customer License and ISV License details 

In [59]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [60]:

import os
try:
    amlWBSharedDir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']    
except:
    print( 'Not in amlwb? define amlWBSharedDir as "/shared_folder_on_host/amlwb_exp_acc/amlwb_work_space/amlwb_experiment"')
    
amlWBSharedDir

'/azureml-share/'

In [61]:
# # Use the Azure Machine Learning data collector to log various metrics
# from azureml.logging import get_azureml_logger
# logger = get_azureml_logger()

In [62]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [63]:
# import utlity functions

import sys, os
paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['Code',  'src'])))]
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils
import azure_chestxray_keras_utils

[None]

In [64]:
# create the file path variables 
# paths are tipically container level dirs mapped to a host dir for data persistence.

prj_consts = azure_chestxray_utils.chestxray_consts()

data_base_input_dir=os.path.join(amlWBSharedDir, 
                                 os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
data_base_output_dir=os.path.join(amlWBSharedDir, 
                                  os.path.join(*(prj_consts.BASE_OUTPUT_DIR_list)))

In [65]:
# global variables

#location of trained models weights, quality will be dependent on train data size
# and number of epochs among other things
weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.MODEL_WEIGHTS_DIR_list))) 
weights_dir
!ls -l {weights_dir}

# "quality" models, fully trained on all training data
fully_trained_weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.FULLY_PRETRAINED_MODEL_DIR_list))) 
fully_trained_weights_dir
!ls -l {fully_trained_weights_dir}


nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))

data_partitions_dir=os.path.join(data_base_output_dir, 
                                os.path.join(*(prj_consts.DATA_PARTITIONS_DIR_list)))  
label_path = os.path.join(data_partitions_dir,'labels14_unormalized_cleaned.pickle')
partition_path = os.path.join(data_partitions_dir, 'partition14_unormalized_cleaned.pickle')

'/azureml-share/chestxray/output/weights_tmpdir'

total 0


'/azureml-share/chestxray/output/fully_trained_models'

total 86320
-rw-rw-r-- 1 1003 1003 30097832 Feb 14 04:37 azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5
-rw-r--r-- 1 root root 29143128 Feb 14 04:55 weights_only_azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5
-rw-rw-r-- 1 1003 1003 29142168 Feb  7 06:16 weights_only_azure_chest_xray__14_weights_712split_epoch_029_val_loss_147.7599.hdf5


In [66]:
# # extract and save the weights from a full model

# import keras_contrib
# from keras.models import load_model
# model_file_name = 'azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5'
# model = load_model(os.path.join(fully_trained_weights_dir, model_file_name))
# model.save_weights(os.path.join(fully_trained_weights_dir, 'weights_only_'+model_file_name))

In [67]:
# get long (full path) model file name

all_models=!ls {os.path.join(fully_trained_weights_dir, '*.hdf5')}
all_models
models_file_name= [os.path.join(fully_trained_weights_dir, 
                               'weights_only_azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5')]

['/azureml-share/chestxray/output/fully_trained_models/azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5',
 '/azureml-share/chestxray/output/fully_trained_models/weights_only_azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5',
 '/azureml-share/chestxray/output/fully_trained_models/weights_only_azure_chest_xray__14_weights_712split_epoch_029_val_loss_147.7599.hdf5']

In [68]:
import os
import pickle

import cv2
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.utils import Sequence
from sklearn import metrics

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

from tensorflow.python.client import device_lib

resized_height = 224
resized_width = 224
num_channel = 3
num_classes = 14
batch_size = 512 #512

In [69]:
def get_available_gpus():
    """
    Returns: number of GPUs available in the system
    """
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()
# get number of available GPUs
print("num of GPUs:", len(get_available_gpus()))

['/device:GPU:0', '/device:GPU:1']

num of GPUs: 2


In [70]:
# device_lib.list_local_devices()
# !nvidia-smi

In [71]:
num_gpu = get_available_gpus()
# get number of available GPUs
print("num of GPUs:", len(get_available_gpus()))

pathologies_name_list = prj_consts.DISEASE_list
pathologies_name_list

stanford_result = [0.8094, 0.9248, 0.8638, 0.7345, 0.8676, 0.7802, 0.7680, 0.8887, 0.7901, 0.8878, 0.9371, 0.8047,
                   0.8062, 0.9164]


with open(label_path, 'rb') as f:
    labels = pickle.load(f)

with open(partition_path, 'rb') as f:
    partition = pickle.load(f)

num of GPUs: 2


['Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax',
 'Consolidation',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Pleural Thickening',
 'Hernia']

In [72]:

# generator for train and validation data
# use the Sequence class per issue https://github.com/keras-team/keras/issues/1638
class DataGenSequence(Sequence):
    def __init__(self, labels, image_file_index, current_state):
        self.batch_size = batch_size
        self.labels = labels
        self.img_file_index = image_file_index
        self.current_state = current_state
        self.len = len(self.img_file_index) // self.batch_size
        print("for DataGenSequence", current_state, "total rows are:", len(self.img_file_index), ", len is", self.len)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        # print("loading data segmentation", idx)
        # make sure each batch size has the same amount of data
        current_batch = self.img_file_index[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, resized_height, resized_width, num_channel))
        y = np.empty((self.batch_size, num_classes))

        for i, image_name in enumerate(current_batch):
            path = os.path.join(nih_chest_xray_data_dir, image_name)

            # loading data

            img = cv2.resize(cv2.imread(path), (resized_height, resized_width)).astype(np.float32)
            X[i, :, :, :] = img
            y[i, :] = labels[image_name]

            # only do random flipping in training status
        if self.current_state == 'train':
            # this is different from the training code
            x_augmented = X
        else:
            x_augmented = X

        return x_augmented, y

In [73]:
len(partition['test'])

32893

In [74]:
import keras_contrib

# load test data
X_test = np.empty((len(partition['test']), 224, 224, 3), dtype=np.float32)
y_test = np.empty((len(partition['test']) - len(partition['test']) % batch_size, 14), dtype=np.float32)

for i, npy in enumerate(partition['test']):
    if (i < len(y_test)):
        # round to batch_size
        y_test[i, :] = labels[npy]

print("len of result is", len(y_test))
y_pred_list = np.empty((len(models_file_name), len(partition['test']), 14), dtype=np.float32)

# individual models
for index, current_model_file in enumerate(models_file_name):
    print(current_model_file)
#    model = load_model(current_model_file)
    model = azure_chestxray_keras_utils.build_model(keras_contrib.applications.densenet.DenseNetImageNet121); model.load_weights(current_model_file)

    print('evaluation for model', current_model_file)
    # y_pred = model.predict(X_test)

    y_pred = model.predict_generator(generator=DataGenSequence(labels, partition['test'], current_state='test'),
                                     workers=32, verbose=1, max_queue_size=1)
    print("result shape", y_pred.shape)
    
    # add one fake row of ones in both test and pred values to avoid:
    # ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.
    y_test = np.insert(y_test, 0, np.ones((y_test.shape[1],)), 0)
    y_pred = np.insert(y_pred, 0, np.ones((y_pred.shape[1],)), 0)

    df = pd.DataFrame(columns=['Disease', 'Our AUC Score', 'Stanford AUC Score'])
    for d in range(14):
        df.loc[d] = [pathologies_name_list[d],
                     metrics.roc_auc_score(y_test[:, d], y_pred[:, d]),
                     stanford_result[d]]

    df['Delta'] = df['Stanford AUC Score'] - df['Our AUC Score']
    df.to_csv(current_model_file + ".csv", index=False)
    print(df)

len of result is 32768
/azureml-share/chestxray/output/fully_trained_models/weights_only_azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5
Weights for the model were loaded successfully
evaluation for model /azureml-share/chestxray/output/fully_trained_models/weights_only_azure_chest_xray_14_weights_712split_epoch_054_val_loss_191.2588.hdf5
for DataGenSequence test total rows are: 32893 , len is 64
64/64 [==============================] - 469s 7s/step
result shape (32768, 14)
               Disease  Our AUC Score  Stanford AUC Score     Delta
0          Atelectasis       0.823191              0.8094 -0.013791
1         Cardiomegaly       0.933519              0.9248 -0.008719
2             Effusion       0.883184              0.8638 -0.019384
3         Infiltration       0.744561              0.7345 -0.010061
4                 Mass       0.859510              0.8676  0.008090
5               Nodule       0.783997              0.7802 -0.003797
6            Pneumonia 

In [78]:
y_pred_list.shape
type(y_test[:, d])
y_test.shape
y_test[:3,]
y_test.sum(axis=0).astype(int)


(1, 32893, 14)

numpy.ndarray

(32769, 14)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
      dtype=float32)

array([4004, 1189, 5187, 7044, 2089, 2090,  593, 2450, 1850,  808,  933,
        493,    1,   79])

In [76]:
# C:\repos\ChestXRay\TDSP>jupyter nbconvert --to html .\Code\02_Model\020_evaluate.ipynb
# [NbConvertApp] Converting notebook .\Code\01_DataPrep\001_get_data.ipynb to html
# [NbConvertApp] Writing 263414 bytes to .\Code\01_DataPrep\001_get_data.html